<a href="https://colab.research.google.com/github/JamieBali/MinesweeperNetworks/blob/main/Minesweeper_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Minesweeper

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random as rand
import math

from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
from sklearn.model_selection import train_test_split

## Grid Construction

In [2]:
def construct_grid(size = (16,16), mines = 40):
  grid = np.zeros(size)
  total = 0
  while total < mines:
    x_loc = rand.randint(0, size[0]-1)
    y_loc = rand.randint(0, size[1]-1)
    if not(x_loc > 1 or y_loc > 1):
      continue
    if grid[x_loc][y_loc] == 9:
      continue
    grid[x_loc][y_loc] = 9
    total += 1

  for x in range(0,size[0]):
    for y in range(0,size[1]):
      if grid[x][y] == 9:
        continue
      minecount = 0
      for x_offset in range(-1,2):
        for y_offset in range(-1,2):
          if 0 <= (x+x_offset) < size[0] and 0 <= (y+y_offset) < size[1]:
            if grid[x+x_offset][y+y_offset] == 9:
              minecount += 1
      grid[x][y] = minecount
  return grid

def click(grid, mask, x_click, y_click):
  if mask[x_click][y_click] == 1:
    return mask
  if grid[x_click][y_click] == 9:
    return None
  mask[x_click][y_click] = 1
  if grid[x_click][y_click] == 0:
    for x in range(-1,2):
      for y in range(-1,2):
        if (x != 0 or y != 0) and (0 <= x_click + x < len(grid)) and (0 <= y_click + y < len(grid[0])):
          mask = click(grid, mask, x_click + x, y_click + y)
  else:
    mask[x_click][y_click] = 1
  return mask

def flag(grid, mask, x_flag, y_flag):
  if mask[x_flag][y_flag] == 1:
    return mask
  if grid[x_flag][y_flag] != 9:
    return None
  mask[x_flag][y_flag] = 1
  return mask
  
def display(grid, mask):
  for x in range(len(grid)):
    outstring = ""
    for y in range(len(grid[x])):
      if mask[x][y] == 0:
        outstring += "■ "
      else:
        outstring += str(grid[x][y])[0] + " "
    print(outstring)

gr = construct_grid()
#msk = np.zeros((16,16))
#msk = click(gr, msk, 0, 0)
#display(gr, msk)
print(gr)

[[0. 0. 1. 9. 1. 0. 0. 1. 1. 2. 9. 9. 2. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 1. 9. 3. 4. 9. 3. 1. 1. 0.]
 [1. 1. 2. 1. 1. 0. 1. 2. 2. 2. 9. 3. 3. 9. 2. 1.]
 [1. 9. 2. 9. 1. 0. 1. 9. 1. 1. 2. 9. 3. 2. 2. 9.]
 [2. 2. 3. 2. 2. 1. 2. 2. 1. 0. 1. 3. 9. 2. 1. 1.]
 [9. 2. 2. 9. 1. 1. 9. 2. 1. 2. 1. 3. 9. 3. 1. 1.]
 [2. 9. 2. 1. 1. 1. 2. 3. 9. 2. 9. 3. 2. 3. 9. 1.]
 [2. 2. 2. 1. 1. 1. 1. 9. 2. 2. 1. 2. 9. 3. 2. 2.]
 [2. 9. 2. 2. 9. 1. 1. 1. 1. 1. 1. 2. 1. 2. 9. 1.]
 [9. 3. 3. 9. 2. 2. 1. 1. 0. 1. 9. 1. 0. 1. 1. 1.]
 [2. 9. 3. 2. 1. 2. 9. 3. 1. 2. 1. 1. 0. 0. 0. 0.]
 [1. 2. 9. 1. 0. 3. 9. 5. 9. 2. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 2. 9. 5. 9. 3. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 2. 9. 3. 9. 2. 1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 2. 1. 2. 9. 1. 0. 1. 9.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1.]]


In [3]:
msk = click(gr, msk, 0, 2)
display(gr, msk)

NameError: ignored

# Design 1 - Full-Grid with Probabilities

In [13]:
class fullGridProbabilitySolver():

  model = None

  def __init__(self, gridsize = (16,16), mines = 40, epochs = 10, testing_training_size = 1000, split = 0.2, batch_size = 64, training_obfuscation = 0.5, verbose = False):
    self.gridsize = gridsize
    self.mines = mines
    self.epochs = epochs
    self.split = split
    self.batch_size = batch_size
    self.testing_training_size = testing_training_size
    self.obfuscated_tiles = math.floor(mines * training_obfuscation)
    self.verbose = verbose

    self.testing_puzzles = []

    self.model = keras.models.Sequential()
    self.model.add(Conv2D((self.gridsize[0]*self.gridsize[1]), kernel_size=(5,5), activation='relu', padding='same', input_shape=(self.gridsize[0],self.gridsize[1],1)))
    self.model.add(BatchNormalization())
    self.model.add(Conv2D((self.gridsize[0]*self.gridsize[1]), kernel_size=(5,5), activation='relu', padding='same'))
    self.model.add(BatchNormalization())
    self.model.add(Conv2D((self.gridsize[0]*self.gridsize[1]), kernel_size=(3,3), activation='relu', padding='same'))

    self.model.add(Flatten())
    self.model.add(Dense((self.gridsize[0]*self.gridsize[1]*2)))
    self.model.add(Reshape((-1,2)))
    self.model.add(Activation('softmax'))

    self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

  def get_data(self):
    data = []
    print("genearting " + str(self.testing_training_size) + " grids")
    for i in range(self.testing_training_size):
      data.append(construct_grid(self.gridsize,self.mines))
      if (i % 100 == 0):
        print("Generated " + str(i) + " puzzles")
    return data

  def normalise(self, val):
    return val/9

  def denormalise(self, val):
    return val*9

  def obfuscate_grid(self, grid):
    for i in range(self.obfuscated_tiles):
      x_loc = rand.randint(0, self.gridsize[0]-1)
      y_loc = rand.randint(0, self.gridsize[1]-1)
      grid[x_loc][y_loc] = 0
    return grid

  def format_as_solution(self, grid):
    binary_solution = (grid == 9)
    return np.array(binary_solution)

  def get_testing_training_split(self):
    data = self.get_data()
    solutions = []
    puzzles = []
    print("working through testing puzzles")
    for puzzle in data:
      solutions.append(self.format_as_solution(np.copy(puzzle)))
      puzzles.append(self.normalise(np.array(self.obfuscate_grid(np.copy(puzzle)))))
    training_puzzles, _, training_solutions, _ = train_test_split(np.array(puzzles), np.array(solutions), test_size=self.split, shuffle=False)
    testing_puzzles = []
    for x in range(1,math.ceil(self.split * self.testing_training_size)):
      testing_puzzles.append(data[x * -1])
    print(str(len(training_puzzles)) + " training puzzles")
    print(str(len(testing_puzzles)) + " testing puzzles")
    return training_puzzles, training_solutions, testing_puzzles


  def train(self, tr_puzzles, tr_solutions):
    self.model.fit(tr_puzzles, tr_solutions, batch_size = self.batch_size, epochs=self.epochs, verbose=1)

  def get_model(self):
    return self.model

  def generate_model(self):
    training_puzzles, training_solutions, testing_puzzles = self.get_testing_training_split()
    self.testing_puzzles = testing_puzzles
    self.training_puzzles = training_puzzles
    training_puzzles = [training_puzzles]
    self.train(training_puzzles, training_solutions)

  def get_validity_mask(self, mask): 
    def is_adjacent(mask, x_coord, y_coord):
      for x_itt in range(-1,2):
        for y_itt in range(-1,2):
          if (0 <= x_coord + x_itt < self.gridsize[0]) and (0 <= y_coord + y_itt < self.gridsize[1]):
            if mask[x_coord + x_itt][y_coord + y_itt] == 1:
              return 1
      return 0
    validity_mask = np.zeros((self.gridsize[0],self.gridsize[1]))
    for row in range(self.gridsize[0]):
      for column  in range(self.gridsize[1]):
        if mask[row][column] == 1:
          validity_mask[row][column] = 0
        else:
          validity_mask[row][column] = is_adjacent(mask, row, column)
    return validity_mask


  def test(self):
    correct = 0
    total = 0
    for puzzle in self.testing_puzzles:
      total += 1
      mask = np.zeros((self.gridsize[0],self.gridsize[1]))
      mask = click(puzzle,mask,0,0)
      while True:
        masked_puzzle = np.copy(puzzle)*mask
        if mask.sum() == self.gridsize[0] * self.gridsize[1]:
          if self.verbose:
            print("Looks like we managed to solve this one")
            print(masked_puzzle)
            print("---------------------------------------")
          correct += 1
          break
        normalised_puzzle = self.normalise(masked_puzzle)
        output = self.model.predict(normalised_puzzle)
        max = np.argmax(output, axis = 2).reshape((self.gridsize[0],self.gridsize[1]))
        probabilities = np.max(output, axis = 2).reshape((self.gridsize[0],self.gridsize[1]))
        probabilities *= self.get_validity_mask(mask)
        index_of_best = np.argmax(probabilities)
        x_locat, y_locat = (index_of_best // self.gridsize[0]), (index_of_best % self.gridsize[1])
        if max[x_locat][y_locat] == 0:
          t_mask = click(puzzle, mask, x_locat, y_locat)
        else:
          t_mask = flag(puzzle, mask, x_locat, y_locat)
        if t_mask:
          mask = t_mask
        else:
          if self.verbose:
            print("Looks like we failed to solve this one")
            display(puzzle, mask)
            print("--------------------------------------")
          break
    print("We were able to solve " + str(correct) + " of the " + str(total) + "training puzzles")
    print("That is an accuracty of " + str((correct/total)*100) + "%")

  def quick_test(self):
    puzzle = self.testing_puzzles[0]
    print(puzzle)
    mask = np.zeros((self.gridsize[0],self.gridsize[1]))
    mask = click(puzzle,mask,0,0)
    display(puzzle, mask)
    while True:
      
      if mask.sum() == self.gridsize[0] * self.gridsize[1]:
        print("Looks like we managed to solve this one")
        display(puzzle,mask)
        print("---------------------------------------")
        break
      normalised_puzzle = self.normalise(np.copy(puzzle))
      masked_puzzle = normalised_puzzle * mask

      output = self.model.predict(np.array([masked_puzzle]))

      max = np.argmax(output, axis = 2).reshape((self.gridsize[0],self.gridsize[1]))
      probabilities = np.max(output, axis = 2).reshape((self.gridsize[0],self.gridsize[1]))
      
      probabilities *= self.get_validity_mask(mask)
      index_of_best = np.argmax(probabilities)

      x_locat, y_locat = (index_of_best // self.gridsize[0]), (index_of_best % self.gridsize[1])
      if max[x_locat][y_locat] == 0:
        t_mask = click(puzzle, mask, x_locat, y_locat)
        print("We picked " + str(x_locat) + "," + str(y_locat))
      else:
        t_mask = flag(puzzle, mask, x_locat, y_locat)
        print("We flagged " + str(x_locat) + "," + str(y_locat))
      if t_mask is None:
        print("Looks like we failed to solve this one")
        display(puzzle, mask)
        print("--------------------------------------")
        break

      mask = t_mask
      print("This is the sate of the board now.")
      display(puzzle, mask)
      print("---------------------------------")

In [16]:
model = fullGridProbabilitySolver(testing_training_size = 1000, epochs=4, batch_size=50, training_obfuscation=0.8)
model.generate_model()
model.quick_test()

genearting 1000 grids
Generated 0 puzzles
Generated 100 puzzles
Generated 200 puzzles
Generated 300 puzzles
Generated 400 puzzles
Generated 500 puzzles
Generated 600 puzzles
Generated 700 puzzles
Generated 800 puzzles
Generated 900 puzzles
working through testing puzzles
800 training puzzles
199 testing puzzles
Epoch 1/4
16/16 [==============================] - 52s 3s/step - loss: 2.6713
Epoch 2/4
16/16 [==============================] - 50s 3s/step - loss: 0.4380
Epoch 3/4
16/16 [==============================] - 50s 3s/step - loss: 0.3111
Epoch 4/4
16/16 [==============================] - 50s 3s/step - loss: 0.1606


[[0. 0. 0. 0. 1. 9. 9. 3. 1. 1. 0. 0. 1. 9. 1. 0.]
 [1. 1. 0. 0. 1. 3. 9. 4. 9. 1. 0. 1. 2. 2. 1. 0.]
 [9. 1. 0. 0. 0. 1. 2. 9. 3. 2. 0. 1. 9. 1. 0. 0.]
 [2. 2. 1. 0. 0. 0. 2. 3. 9. 1. 0. 2. 2. 2. 0. 0.]
 [2. 9. 2. 1. 1. 0. 1. 9. 2. 1. 0. 1. 9. 2. 1. 1.]
 [9. 3. 3. 9. 1. 0. 1. 1. 1. 0. 1. 2. 2. 2. 9. 1.]
 [2. 9. 3. 2. 1. 0. 0. 0. 0. 0. 1. 9. 1. 1. 1. 1.]
 [1. 2. 9. 1. 0. 0. 0. 0. 1. 1. 2. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 0. 1. 1. 2. 2. 9. 1. 0. 0. 0. 1. 9.]
 [1. 1. 2. 1. 1. 1. 9. 2. 9. 2. 1. 0. 0. 0. 1. 1.]
 [2. 9. 3. 9. 1. 2. 2. 3. 2. 2. 2. 2. 2. 2. 1. 1.]
 [3. 9. 3. 1. 2. 3. 9. 2. 1. 9. 3. 9. 9. 2. 9. 1.]
 [9. 3. 2. 1. 1. 9. 9. 2. 1. 3. 9. 4. 3. 3. 2. 1.]
 [2. 3. 9. 1. 1. 2. 2. 1. 0. 2. 9. 3. 2. 9. 1. 0.]
 [9. 2. 1. 1. 0. 0. 1. 1. 1. 1. 2. 9. 2. 1. 1. 0.]
 [1. 1. 0. 0. 0. 0. 1. 9. 1. 0. 1. 1. 1. 0. 0. 0.]]
0 0 0 0 1 ■ ■ ■ ■ 1 0 0 1 ■ ■ ■ 
1 1 0 0 1 3 ■ ■ ■ 1 0 1 2 ■ ■ ■ 
■ 1 0 0 0 1 2 ■ ■ 2 0 1 ■ ■ ■ ■ 
■ 2 1 0 0 0 2 ■ ■ 1 0 2 ■ ■ ■ ■ 
■ ■ 2 1 1 0 1 ■ 2 1 0 1 ■ ■ ■ ■ 
■ ■ ■ ■ 1 0 1 1 1 

In [ ]:
model.testing_puzzles[0]

array([[0., 1., 9., 1., 1., 9., 3., 9., 2., 1., 9., 9., 1., 1., 1., 1.],
       [1., 2., 2., 1., 1., 2., 4., 9., 2., 1., 2., 2., 1., 1., 9., 1.],
       [1., 9., 2., 1., 1., 1., 9., 2., 1., 1., 1., 1., 1., 3., 3., 2.],
       [1., 1., 2., 9., 2., 3., 2., 2., 0., 1., 9., 2., 3., 9., 9., 1.],
       [0., 0., 1., 2., 9., 2., 9., 1., 0., 1., 2., 9., 3., 9., 3., 1.],
       [0., 1., 1., 2., 1., 2., 1., 1., 1., 1., 2., 1., 2., 1., 1., 0.],
       [1., 2., 9., 1., 0., 0., 0., 1., 2., 9., 1., 0., 0., 0., 0., 0.],
       [1., 9., 2., 1., 0., 1., 1., 2., 9., 2., 1., 0., 0., 0., 0., 0.],
       [2., 3., 3., 2., 1., 1., 9., 2., 1., 1., 0., 0., 0., 0., 0., 0.],
       [9., 2., 9., 9., 2., 2., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.],
       [1., 3., 3., 4., 9., 1., 0., 0., 1., 1., 1., 0., 1., 9., 1., 0.],
       [1., 2., 9., 2., 1., 1., 1., 1., 2., 9., 1., 0., 1., 1., 2., 1.],
       [2., 9., 3., 1., 0., 0., 1., 9., 3., 2., 3., 2., 2., 1., 1., 9.],
       [2., 9., 2., 0., 0., 0., 1., 1., 2., 9., 2.,

# Design 2 - Segmented Grid with Probabilities

In [ ]:
var = None
if var:
  print("y")
else:
  print("N")

N
